In [1]:
import os
import sys
import pickle
import random
import warnings
warnings.filterwarnings("ignore")

# add the path of packages to system path
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

# add packages for computation
import numpy as np
import pandas as pd
import marketmodel.loader as loader
import marketmodel.utils as utils

from marketmodel.factors import PrepTrainData, DecodeFactor
from marketmodel.loader import DataHestonSlv
from marketmodel.neuralsde import Train, Simulate, Model

# load configurations
hp_model_S = utils.Config.hp_model_S
hp_model_mu = utils.Config.hp_model_mu
hp_model_xi = utils.Config.hp_model_xi
hp_sde_transform = utils.Config.hp_sde_transform

# set paths
hestonslv_fname = '../input/sim_hestonslv.pkl'
checkpoint_model_S_initial = '../output/checkpoint/initial_model_S/model_S_0'

# Load and pre-process data

In [2]:
# load Heston-SLV simulation data
St, vt, list_exp, list_mny, cs_ts_raw, cs_ts, mask_quality_value, \
Ts, ks, mat_A, vec_b = loader.load_hestonslv_data(hestonslv_fname)

# load the trained initial model for S
X_S, Y_S = PrepTrainData.prep_data_model_S_initial(
    St, cs_ts, max_PC=7, factor_multiplier=1e5)

model_S_initial = Model.construct_S(X_S.shape[1], X_S.shape[0], 
                                    hp_model_S['pruning_sparsity'],
                                    hp_model_S['validation_split'],
                                    hp_model_S['batch_size'],
                                    hp_model_S['epochs'])
model_S_initial.load_weights(checkpoint_model_S_initial)

# calculate derivatives for the normalised call prices
cT_ts, cm_ts, cmm_ts = PrepTrainData.calc_call_derivatives(
    list_mny, list_exp, cs_ts_raw, mask_quality_value)

z_ts = PrepTrainData.calc_zt(cT_ts, cm_ts, cmm_ts, model_S_initial, X_S)

# define the effective constraints through their inequality constraints
mask_redundant = DecodeFactor.find_redundant_constraints(-mat_A, -vec_b)
mat_A_nec = mat_A[~mask_redundant, :]
vec_b_nec = vec_b[~mask_redundant]

Instructions for updating:
Please use `layer.add_weight` method instead.


100%|██████████| 281/281 [00:00<00:00, 323.94it/s]


# Decode factors

In [3]:
# calculate the constant term G0
G0 = cs_ts.mean(axis=0)
res0 = cs_ts - G0[None, :]

## One dynamic arbitrage factor

In [4]:
# parameters
n_da_factor = 1  # number of dynamic arbitrage factor

# decode dynamic arbitrage factors
G_da, xi_da = DecodeFactor.decode_dynarb_factor(
    res0, n_da_factor, model_S_initial, X_S, cT_ts, cm_ts, cmm_ts)

Gx = G_da
X = xi_da

res, mape, psas, pda = DecodeFactor.report_factor_metrics(G0, Gx, X, cs_ts, mat_A_nec, vec_b_nec, z_ts, True)

Residual norm: 0.3072
MAPE:          24.37%
PSAS:          60.67%
PDA:           3.51%


## Two dynamic arbitrage factors

In [5]:
# parameters
n_da_factor = 2  # number of dynamic arbitrage factor

# decode dynamic arbitrage factors
G_da, xi_da = DecodeFactor.decode_dynarb_factor(
    res0, n_da_factor, model_S_initial, X_S, cT_ts, cm_ts, cmm_ts)

Gx = G_da
X = xi_da

res, mape, psas, pda = DecodeFactor.report_factor_metrics(G0, Gx, X, cs_ts, mat_A_nec, vec_b_nec, z_ts, True)

Residual norm: 0.2852
MAPE:          17.22%
PSAS:          69.85%
PDA:           0.75%


## One dynamic arbitrage factor and one static arbitrage factor

In [7]:
# parameters
n_da_factor = 1  # number of dynamic arbitrage factor
n_sa_factor = 1
n_PC = 6

# decode dynamic arbitrage factors
G_da, xi_da = DecodeFactor.decode_dynarb_factor(
    res0, n_da_factor, model_S_initial, X_S, cT_ts, cm_ts, cmm_ts)

res1 = res0 - xi_da.dot(G_da)

# decode static arbitrage factors
rhs0 = vec_b_nec - G0.dot(mat_A_nec.T)
rhs1 = rhs0[:, None] - mat_A_nec.dot(G_da.reshape((-1, 1))). \
    dot(xi_da.reshape((1, -1)))

G_sa, xi_sa, weights = DecodeFactor.decode_stcarb_factor(
    res1, n_sa_factor, n_PC, mat_A_nec, rhs1)

Searching for the static-arbitrage minimisation direction for projection.
****** Py-BOBYQA Results ******
Solution xmin = [ 0.72175876 -0.25943467 -0.27096681 -0.03083228  0.02673118 -0.01994683]
Objective value f(xmin) = -9951.829429
Needed 6350 objective evaluations (at 6350 points)
Did a total of 42 runs
Approximate gradient = [ 13.7996479  -43.54494982  15.61568002  -1.28424445 -39.79832513
  75.306725  ]
Approximate Hessian = [[ 1.76618998e+10 -4.75737772e+09 -5.04413414e+09 -1.57491732e+09
  -1.67914167e+09  7.08978991e+09]
 [-4.75737772e+09  6.93915631e+09 -2.00835463e+09 -8.11150129e+08
   1.65436253e+09 -8.43461281e+09]
 [-5.04413414e+09 -2.00835463e+09  1.33980779e+10  3.06552612e+09
   2.97797379e+09 -6.75036949e+09]
 [-1.57491732e+09 -8.11150129e+08  3.06552612e+09  3.81077529e+09
   3.78088406e+09  2.02116765e+09]
 [-1.67914167e+09  1.65436253e+09  2.97797379e+09  3.78088406e+09
   7.93658315e+09 -5.78279433e+09]
 [ 7.08978991e+09 -8.43461281e+09 -6.75036949e+09  2.0211676

In [9]:
# combine factors
Gx = np.vstack((G_da, G_sa))
X = np.hstack((xi_da, xi_sa))

res, mape, psas, pda = DecodeFactor.report_factor_metrics(G0, Gx, X, cs_ts, mat_A_nec, vec_b_nec, z_ts, True)

Residual norm: 0.1706
MAPE:          4.51%
PSAS:          0.49%
PDA:           2.72%


## One dynamic factor and one statistical accuracy factor

In [10]:
# parameters
n_da_factor = 1  # number of dynamic arbitrage factor
n_pca_factor = 1

# decode dynamic arbitrage factors
G_da, xi_da = DecodeFactor.decode_dynarb_factor(
    res0, n_da_factor, model_S_initial, X_S, cT_ts, cm_ts, cmm_ts)

res1 = res0 - xi_da.dot(G_da)

G_pca, xi_pca = DecodeFactor.decode_pca_factor(res1, n_pca_factor)

# combine factors
Gx = np.vstack((G_da, G_pca))
X = np.hstack((xi_da, xi_pca))

res, mape, psas, pda = DecodeFactor.report_factor_metrics(G0, Gx, X, cs_ts, mat_A_nec, vec_b_nec, z_ts, True)

Residual norm: 0.1129
MAPE:          5.11%
PSAS:          28.11%
PDA:           3.21%


## Two statistical accuracy factors

In [11]:
# parameters
n_pca_factor = 2

G_pca, xi_pca = DecodeFactor.decode_pca_factor(res0, n_pca_factor)

# combine factors
Gx = G_pca
X = xi_pca

res, mape, psas, pda = DecodeFactor.report_factor_metrics(G0, Gx, X, cs_ts, mat_A_nec, vec_b_nec, z_ts, True)

Residual norm: 0.0728
MAPE:          2.84%
PSAS:          39.44%
PDA:           4.46%
